In [1]:
#import modules and set up environment 
import os
import sys
path = "../../src/"

sys.path.append(path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import jax.numpy as jnp
from jax import jit, grad, vmap, hessian, scipy, random

#import sgmcmc code 
import models.bivariate_gaussian.bivariate_gaussian as bvg
import samplers.sgld as sgld
import samplers.sgldps as sgldps
import samplers.sgldcv as sgldcv
import samplers.sgldcvps as sgldcvps

key = random.PRNGKey(0)

### Set up model and data

In [2]:
#load in the data
file_path = "../../data/synthetic/toy_bvg_synth.csv"
data = pd.read_csv(file_path)
dat_array = data.values[:]
x = np.array(dat_array)
N = dat_array.shape[0]
y = np.array([None]*N) 
y.shape

(1000,)

In [3]:
#set up model parameters
dim = 2 
theta_true = jnp.array([0., 1.]) #data mean
xbar = jnp.mean(x, axis = 0)
sigma_x = jnp.array([[ 2*10**5 , -4*10**4], [-4*10**4,  10**4]]) #data covariance matrix
sigma_x_inv = jnp.linalg.inv(sigma_x) #data precision 
mu_0 = jnp.array([0., 0.]) #prior mean
lambda_0 = jnp.array([[ 1000 , 0.], [0.,  1000]])  #prior scale
lambda_0_inv = jnp.linalg.inv(lambda_0)

#posterior 
lambda_1_inv = lambda_0_inv + N*sigma_x_inv #posterior precision
lambda_1 = jnp.linalg.inv(lambda_1_inv) #posterior covariance
mu_1 = jnp.matmul(lambda_1, N*jnp.matmul(sigma_x_inv, xbar) + jnp.matmul(lambda_0_inv, mu_0)) #posterior mean

In [4]:
#define the mode
theta_hat = mu_1 

# draw candidate thetas
n_candidate = 10
theta_candidates = random.multivariate_normal(key=key, mean=mu_1, cov=lambda_1, shape=(n_candidate,))
theta_candidates

DeviceArray([[ -1.12365294,  10.89599973],
             [ 13.91596074,  11.07020949],
             [  1.14693132,  14.07728093],
             [-16.36011229,  17.51262531],
             [  5.18900029,  10.08220814],
             [ -5.58482982,  12.4932415 ],
             [ -2.16542207,  15.96377277],
             [ -6.88311986,  17.80447463],
             [ -5.7467338 ,  13.21220418],
             [-11.68235157,  15.75879915]], dtype=float64)

### Set up experiment 

In [5]:
batch_sizes = jnp.int32(jnp.linspace(0.01, 0.99, num=50)*N)

In [6]:
#set up model-specific gradient functions
gradf_0 = bvg.gradf_0
gradf_i_batch = bvg.gradf_i_batch
post_var = bvg.post_var

In [7]:
pseudo_var = dict()
reps = 1000

### SGLD gradients (with replacement)

In [8]:
pseudo_var['sgld-wr'] = np.zeros((n_candidate, batch_sizes.shape[0])) 
#sgld (wr) gradients
for i in tqdm(range(n_candidate), desc = "Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgld_gradients1 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgld.sgld_grad(subkey, theta_candidate, gradf_0, gradf_i_batch, x, y, n, replacement = True)
            sgld_gradients1[k] = grad
        pseudo_var['sgld-wr'][i,j] = np.sum(np.var(sgld_gradients1, axis = 0))

Theta candidates tested: 100%|██████████| 10/10 [06:09<00:00, 36.97s/it]


### SGLD gradients (without replacement)

In [9]:
pseudo_var['sgld-wor'] = np.zeros((n_candidate, batch_sizes.shape[0])) 
#sgld (wor) gradients
for i in tqdm(range(n_candidate), desc = "Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgld_gradients2 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgld.sgld_grad(subkey, theta_candidate, gradf_0, gradf_i_batch, x, y, n,replacement = False)
            sgld_gradients2[k] = grad
        pseudo_var['sgld-wor'][i,j] = np.sum(np.var(sgld_gradients2, axis = 0))

Theta candidates tested: 100%|██████████| 10/10 [15:09<00:00, 90.94s/it]


### SGLD-PS (exact probabilities)

In [10]:
#sgld-ps (exact) gradients
pseudo_var['sgld-ps-exact'] = np.zeros((n_candidate, batch_sizes.shape[0])) 

In [12]:
for i in tqdm(range(n_candidate), desc = "Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    probs1 = sgldps.exact_probs(theta_candidate, gradf_i_batch, x, y)
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgldps_gradients1 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgldps.sgldps_grad(subkey, theta_candidate, gradf_0, gradf_i_batch, probs1, x, y, n)
            sgldps_gradients1[k] = grad
        pseudo_var['sgld-ps-exact'][i,j] = np.sum(np.var(sgldps_gradients1, axis = 0))

Theta candidates tested: 100%|██████████| 10/10 [17:15<00:00, 103.52s/it]


### SGLD-PS (approx)

In [13]:
#sgld-ps (approx) gradients
f_i_grad_list = sgldps.ps_preliminaries(theta_hat, gradf_i_batch, x, y)
probs2 = sgldps.approx_probs(theta_hat, f_i_grad_list)
pseudo_var['sgld-ps-approx'] = np.zeros((n_candidate, batch_sizes.shape[0])) 

In [14]:
for i in tqdm(range(n_candidate), desc = "Theta candidates tested"):
    theta_candidate = theta_candidates[i]
    for j in range(batch_sizes.shape[0]):
        n = batch_sizes[j]
        sgldps_gradients2 = np.zeros((reps, dim))
        for k in range(reps):
            key, subkey = random.split(key)
            grad = sgldps.sgldps_grad(subkey, theta_candidate, gradf_0, gradf_i_batch, probs2, x, y, n)
            sgldps_gradients2[k] = grad
        pseudo_var['sgld-ps-approx'][i, j] = np.sum(np.var(sgldps_gradients2, axis = 0))

Theta candidates tested: 100%|██████████| 10/10 [15:57<00:00, 95.76s/it]


### Save results 

In [15]:
plot_data = []
for key in pseudo_var:
    smooth_path = pseudo_var[key].mean(axis = 0)
    plot_data.append(smooth_path)
    path_deviation = 2*pseudo_var[key].std(axis = 0)
    under_line = (smooth_path-path_deviation)
    plot_data.append(under_line)
    over_line = (smooth_path+path_deviation)
    plot_data.append(over_line) 
    
df = pd.DataFrame(plot_data).T
df['proportion'] = np.linspace(0.01, 0.99, num = 50)

In [16]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,proportion
0,4.286642e+10,3.910150e+10,4.663135e+10,4.333666e+10,3.959271e+10,4.708062e+10,2.949397e+10,2.782444e+10,3.116350e+10,2.941073e+10,2.705541e+10,3.176606e+10,0.01
1,1.426413e+10,1.328303e+10,1.524524e+10,1.451506e+10,1.379958e+10,1.523054e+10,9.588912e+09,8.971133e+09,1.020669e+10,9.804989e+09,8.963483e+09,1.064650e+10,0.03
2,8.874715e+09,8.178782e+09,9.570647e+09,8.177979e+09,7.705573e+09,8.650384e+09,5.914396e+09,5.533784e+09,6.295007e+09,5.967556e+09,5.621921e+09,6.313192e+09,0.05
3,6.415751e+09,6.060470e+09,6.771032e+09,5.896994e+09,5.515039e+09,6.278949e+09,4.266415e+09,3.877600e+09,4.655230e+09,4.284063e+09,4.070769e+09,4.497357e+09,0.07
4,4.781324e+09,4.437021e+09,5.125628e+09,4.424186e+09,4.063527e+09,4.784845e+09,3.232648e+09,3.090673e+09,3.374624e+09,3.387598e+09,3.177728e+09,3.597469e+09,0.09
5,3.943642e+09,3.572115e+09,4.315170e+09,3.500167e+09,3.246913e+09,3.753421e+09,2.743706e+09,2.581850e+09,2.905561e+09,2.651712e+09,2.475610e+09,2.827814e+09,0.11
6,3.367711e+09,3.107039e+09,3.628383e+09,2.896790e+09,2.711896e+09,3.081684e+09,2.223470e+09,2.105141e+09,2.341800e+09,2.247786e+09,2.094718e+09,2.400853e+09,0.13
7,2.866736e+09,2.596438e+09,3.137033e+09,2.535390e+09,2.330624e+09,2.740156e+09,1.978673e+09,1.777719e+09,2.179628e+09,1.936755e+09,1.839984e+09,2.033526e+09,0.15
8,2.563694e+09,2.415863e+09,2.711525e+09,2.157974e+09,2.075493e+09,2.240456e+09,1.767786e+09,1.588929e+09,1.946644e+09,1.737485e+09,1.652560e+09,1.822410e+09,0.17
9,2.287941e+09,2.151275e+09,2.424607e+09,1.838647e+09,1.699153e+09,1.978141e+09,1.540613e+09,1.459801e+09,1.621425e+09,1.555159e+09,1.449663e+09,1.660654e+09,0.19


In [17]:
path_out = "./out/toy_bvg_gradient_comp.csv"
df.to_csv(path_out, index = False) #save csv